In [1]:
def list_bucket_contents(bucket, match=''):
    bucket_resource = s3_resource.Bucket(bucket)
    for key in bucket_resource.objects.all():
        if match in key.key:
            print(key.key)

In [2]:
list_bucket_contents(bucket='nyc-tlc',
                     match='2020')

NameError: name 's3_resource' is not defined